In [1]:
import pandas as pd
import numpy as np

In [57]:
df=pd.read_csv('train_prep.csv')

In [93]:
features=df.drop('Loan Sanction Amount (USD)',axis=1)
label=df['Loan Sanction Amount (USD)']
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler,MinMaxScaler,StandardScaler
xtrain,xtest,ytrain,ytest=train_test_split(features,label,random_state=42,test_size=0.33)

In [94]:
sc=RobustScaler()
xtrain=sc.fit_transform(xtrain)
xtest=sc.transform(xtest)

In [98]:
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
param=[{'n_estimators':np.arange(70,150,10),'max_features':['log2','sqrt','auto'],'max_depth':np.arange(10,40,5)},
        {'n_estimators':np.arange(70,150,10),'max_features':['log2','sqrt','auto'],'max_depth':[None]}]
rcv=RandomizedSearchCV(estimator=RandomForestRegressor(),param_distributions=param,return_train_score=True,scoring='r2')
rcv.fit(xtrain,ytrain)

RandomizedSearchCV(estimator=RandomForestRegressor(),
                   param_distributions=[{'max_depth': array([10, 15, 20, 25, 30, 35]),
                                         'max_features': ['log2', 'sqrt',
                                                          'auto'],
                                         'n_estimators': array([ 70,  80,  90, 100, 110, 120, 130, 140])},
                                        {'max_depth': [None],
                                         'max_features': ['log2', 'sqrt',
                                                          'auto'],
                                         'n_estimators': array([ 70,  80,  90, 100, 110, 120, 130, 140])}],
                   return_train_score=True, scoring='r2')

In [99]:
rcv.best_params_

{'n_estimators': 90, 'max_features': 'auto', 'max_depth': 35}

In [100]:
rf=RandomForestRegressor(n_estimators=90,max_features='auto',max_depth=35)
rf.fit(xtrain,ytrain)

RandomForestRegressor(max_depth=35, n_estimators=90)

In [101]:
pred=rf.predict(xtest)

In [102]:
from sklearn.metrics import r2_score
print(r2_score(ytest,pred))

0.7336096572713395


In [104]:
train=df.drop('Loan Sanction Amount (USD)',axis=1)
label=df['Loan Sanction Amount (USD)']
sc=RobustScaler()
train=sc.fit_transform(train)
rf=RandomForestRegressor(n_estimators=90,max_features='auto',max_depth=35) # yielded r2 of 81.10054 --> RandomForest
rf.fit(train,label)

RandomForestRegressor(max_depth=35, n_estimators=90)

In [105]:
# test---
test=pd.read_csv('test_prep.csv')
test=test[df.columns[:-1]]

In [106]:
test=sc.transform(test)

In [107]:
pred=rf.predict(test)

In [108]:
submit=pd.read_csv('test.csv')

submit=submit['Customer ID']
submit=pd.DataFrame(submit)

submit['Loan Sanction Amount (USD)']=pred

submit

,Customer ID,Loan Sanction Amount (USD)
0,C-26247,78221.727000
1,C-35067,62939.832556
2,C-34590,1342.895889
3,C-16668,67680.870778
4,C-12196,65856.688444
...,...,...
19995,C-9076,63357.806111
19996,C-17587,29480.950618
19997,C-46479,0.000000
19998,C-3099,72601.013667


In [109]:
submit.to_csv('submit2.csv',index=False)